In [149]:
import pandas as pd
import json
import os

from decouple import config
import pyodbc
import numpy as np

In [150]:
data = pd.read_csv('water_data.csv')

In [151]:
data_unpivot = data.melt(id_vars=['ID', 'INTERFACE_ID', 'DATETIME'], var_name='aid', value_name='value')
data_unpivot['aid'] = data_unpivot['aid'].map(lambda x: x.lstrip('AI'))
data_unpivot = data_unpivot.rename(columns={"INTERFACE_ID" : "unitID", "aid" : "analogID", "value" : "readingValue"})
data_unpivot = data_unpivot.astype({'analogID': 'int64'})

In [152]:
#data_unpivot.to_csv('water_data_unpivot.csv', index=False)

In [153]:
BASE_URL = config('SEL_API_URL')
USER_KEY = config('SEL_USER_KEY')
API_KEY = config('SEL_API_KEY')

DB_DRIVER = config('DB_DRIVER')
DB_URL = config('AZURE_DB_SERVER')
DB_BATABASE = config('AZURE_DB_DATABASE')
DB_USR = config('AZURE_DB_USR')
DB_PWD = config('AZURE_DB_PWD')

# Formatted connection string for the SQL DB.
SQL_CONN_STR = "DRIVER={0};SERVER={1};Database={2};UID={3};PWD={4};".format(DB_DRIVER, DB_URL, DB_BATABASE, DB_USR, DB_PWD)

In [154]:
conn = pyodbc.connect(SQL_CONN_STR)

In [155]:
sql = """
		SELECT DISTINCT [unitGUID], [unitID], [unitName]
		FROM [SEL_UNITS]
	"""
units = pd.read_sql(sql, conn)

In [156]:
sql = """
		SELECT DISTINCT r.[sensorGUID], r.[unitGUID], r.[analogID], r.[mUnitGUID], s.sensorName
		FROM [SEL_READINGS] as r
		JOIN [SEL_SENSORS] as s
			ON (r.[sensorGUID] = s.[sensorGUID])
	"""
sensors = pd.read_sql(sql, conn)

In [157]:
sql = """
		SELECT DISTINCT [mUnitGUID], [mUnitName]
		FROM [SEL_MEASURE_UNITS]
	"""
m_units = pd.read_sql(sql, conn)

In [158]:
# Find and process IDs here
join_df = data_unpivot.merge(units, on="unitID", how="inner")

In [159]:
join_df = join_df.merge(sensors, on=["unitGUID", "analogID"])

In [160]:
join_df = join_df.merge(m_units, on=["mUnitGUID"])

In [162]:
join_df

,ID,unitID,DATETIME,analogID,readingValue,unitGUID,unitName,sensorGUID,mUnitGUID,sensorName,mUnitName
0,10449866,527,2021-05-28 01:57:08,0,58.6,452E6EF6-2BD0-4AD0-8D7C-C21A325327F7,Control Panel 4 PH1,61743A91-4C9A-42AA-86DE-649AA30CB680,C19BEB92-BD1D-49D9-8B25-02DA1CCF3819,Moisture Sensor 1,%
1,10449878,527,2021-05-28 01:58:09,0,58.7,452E6EF6-2BD0-4AD0-8D7C-C21A325327F7,Control Panel 4 PH1,61743A91-4C9A-42AA-86DE-649AA30CB680,C19BEB92-BD1D-49D9-8B25-02DA1CCF3819,Moisture Sensor 1,%
2,10449891,527,2021-05-28 01:59:09,0,58.6,452E6EF6-2BD0-4AD0-8D7C-C21A325327F7,Control Panel 4 PH1,61743A91-4C9A-42AA-86DE-649AA30CB680,C19BEB92-BD1D-49D9-8B25-02DA1CCF3819,Moisture Sensor 1,%
3,10449903,527,2021-05-28 02:00:11,0,58.6,452E6EF6-2BD0-4AD0-8D7C-C21A325327F7,Control Panel 4 PH1,61743A91-4C9A-42AA-86DE-649AA30CB680,C19BEB92-BD1D-49D9-8B25-02DA1CCF3819,Moisture Sensor 1,%
4,10449915,527,2021-05-28 02:01:09,0,58.6,452E6EF6-2BD0-4AD0-8D7C-C21A325327F7,Control Panel 4 PH1,61743A91-4C9A-42AA-86DE-649AA30CB680,C19BEB92-BD1D-49D9-8B25-02DA1CCF3819,Moisture Sensor 1,%
...,...,...,...,...,...,...,...,...,...,...,...
5758575,13588442,526,2021-11-15 12:35:58,0,289.0,6FF33116-2BA9-4217-9B38-777F1A03B95D,Control Panel 2 PH1,FC273DCD-9517-4023-B0E0-340AFDAB54FC,AE66ACEE-EC04-4F20-9B13-DC664316A1BC,Level 1,mm
5758576,13588455,526,2021-11-15 12:36:57,0,290.0,6FF33116-2BA9-4217-9B38-777F1A03B95D,Control Panel 2 PH1,FC273DCD-9517-4023-B0E0-340AFDAB54FC,AE66ACEE-EC04-4F20-9B13-DC664316A1BC,Level 1,mm
5758577,13588469,526,2021-11-15 12:37:57,0,289.0,6FF33116-2BA9-4217-9B38-777F1A03B95D,Control Panel 2 PH1,FC273DCD-9517-4023-B0E0-340AFDAB54FC,AE66ACEE-EC04-4F20-9B13-DC664316A1BC,Level 1,mm
5758578,13588482,526,2021-11-15 12:38:57,0,289.0,6FF33116-2BA9-4217-9B38-777F1A03B95D,Control Panel 2 PH1,FC273DCD-9517-4023-B0E0-340AFDAB54FC,AE66ACEE-EC04-4F20-9B13-DC664316A1BC,Level 1,mm


In [163]:
join_df = join_df[['DATETIME', 'analogID', 'readingValue', 'unitName', 'sensorName', 'mUnitName']].copy()

In [ ]:
join_df.to_csv('water_data_join.csv', index=False)

In [104]:
new_df = join_df[["unitGUID","mUnitGUID", "sensorGUID", "analogID", "readingValue"]].copy()

In [105]:
cursor = conn.cursor()
cursor.fast_executemany = True

In [106]:
sql = """
		SET NOCOUNT ON;
		DECLARE @readingGUID UNIQUEIDENTIFIER
		SET @readingGUID = NULL
		SET @readingGUID = NEWID()
		INSERT INTO [SEL_READINGS] (readingGUID, unitGUID, mUnitGUID, sensorGUID, analogID, readingValue) 
		OUTPUT Inserted.readingGUID
		VALUES (@readingGUID, ?, ?, ?, ?, ?);
	"""
params = new_df.values.tolist()
unitGUID = cursor.executemany(sql, params)

OperationalError: ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: An existing connection was forcibly closed by the remote host.\r\n (10054) (SQLExecute); [08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (10054)')

In [ ]:
try:
	first_result = cursor.fetchall()
except pyodbc.ProgrammingError:
	first_result = None

result_sets = []
while cursor.nextset():
	result_sets.append(cursor.fetchall())
all_inserted_ids = np.array(result_sets).flatten()

In [ ]:
cursor.commit()

In [ ]:
sql = """
		INSERT INTO [SEL_UPDATES] (readingGUID, lastUpdate) VALUES (?, ?)
	"""
params = (unitGUID, data_unpivot[['DATETIME']])
cursor.executemany(sql, params)

In [ ]:
try:
	first_result = cursor.fetchall()
except pyodbc.ProgrammingError:
	first_result = None

result_sets = []
while cursor.nextset():
	result_sets.append(cursor.fetchall())
all_inserted_ids = np.array(result_sets).flatten()

In [ ]:
cursor.commit()

In [ ]:
cursor.close()
conn.close()